
# TradingView Strategy Backtest Playground

Notebook ini memuat pipeline lengkap untuk menguji sinyal strategi yang diekspor dari TradingView. Ia memandu mulai dari memuat data CSV, mengadaptasikannya ke format QF-Lib, menjalankan backtest, hingga analisis trade kalah dan eksperimen optimasi parameter tambahan.



> **Struktur notebook**
>
> 1. Parameter input & pemuatan data
> 2. Adaptasi data ke QF-Lib
> 3. Strategi berbasis sinyal
> 4. Menjalankan backtest & mengekstrak hasil
> 5. Visualisasi
> 6. Analisis trade kalah & investigasi
> 7. Eksplor optimasi parameter
> 8. Dokumentasi & reusable structure


In [1]:
from __future__ import annotations

import sys
from pathlib import Path
from typing import Dict, Iterable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
plt.switch_backend('Agg')

plt.style.use("seaborn-v0_8-darkgrid")

PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
if str(PROJECT_ROOT / 'src') not in sys.path:
    sys.path.append(str(PROJECT_ROOT / 'src'))

from src.strategy_backtest import (
    SignalBacktester,
    get_strategy,
    list_strategies,
    load_strategy_csv,
)


## 1. Parameter input & pemuatan data

In [2]:
DATA_FILE = PROJECT_ROOT / 'data' / 'OKX_BTCUSDT, 1D.csv'
TIME_COLUMN = 'time'
PRICE_COLUMN = 'close'
ASSET_SYMBOL = 'BTCUSDT'

STRATEGY_NAME = 'ema112_atr'
STRATEGY_KWARGS = {
    'slow_span': 112,
    'atr_window': 14,
    'atr_multiplier': 1.5,
    'risk_reward': 2.0,
    'trend_lookback': 5,
}

OPTIMISATION_GRID = {
    'atr_multiplier': [1.0, 1.3, 1.5, 1.8],
    'risk_reward': [1.5, 2.0, 2.5],
}


In [3]:

data, column_mapping = load_strategy_csv(DATA_FILE, time_column=TIME_COLUMN)
print(f'Dataset berisi {len(data):,} bar dengan {len(data.columns)} kolom.')
print('Contoh mapping kolom (sanitised -> original):')
for alias, original in list(column_mapping.items())[:10]:
    print(f'  {alias} -> {original}')

data.head()


Dataset berisi 300 bar dengan 48 kolom.
Contoh mapping kolom (sanitised -> original):
  time -> time
  open -> open
  high -> high
  low -> low
  close -> close
  ema -> EMA
  lucid_connector -> LUCID Connector
  internal_higher_high -> Internal Higher High
  internal_lower_high -> Internal Lower High
  internal_lower_low -> Internal Lower Low


,open,high,low,close,ema,lucid_connector,internal_higher_high,internal_lower_high,internal_lower_low,internal_higher_low,...,upper_confluence_zone,lower_confluence_zone,confluence_meter_value,custom_alert_condition_highlighter_1,alert_scripting_condition_highlighter_1,at_valuewhen_1,atr,histogram,macd,signal
time,,,,,,,,,,,,,,,,,,,,,
2025-01-17,99990.9,105847.4,99950.1,104084.4,87477.819959,2201170739202,NaN,NaN,NaN,NaN,...,105,-5,71.428571,NaN,NaN,NaN,3858.413818,761.135818,806.234733,45.098916
2025-01-18,104084.5,104984.0,102288.0,104551.6,87780.010756,524290,NaN,NaN,NaN,NaN,...,105,-5,71.428571,NaN,NaN,NaN,3775.384260,974.775303,1263.568044,288.792741
2025-01-19,104551.6,106448.6,99639.4,101334.0,88019.904371,40962,NaN,NaN,NaN,NaN,...,105,-5,71.428571,NaN,NaN,NaN,3992.085384,849.608478,1350.803339,501.194861
2025-01-20,101334.0,109800.0,99526.4,102264.3,88272.017568,8194,109800.0,NaN,NaN,NaN,...,105,-5,57.142857,NaN,NaN,NaN,4440.765000,781.418731,1477.968275,696.549544
2025-01-21,102263.9,107285.8,100111.0,106150.1,88588.443806,67586,NaN,NaN,NaN,NaN,...,105,-5,57.142857,NaN,NaN,NaN,4636.053214,939.347827,1870.734327,931.386500



## 2. Adaptasi data ke QF-Lib

Data sudah menggunakan `DatetimeIndex` dan kolom harga numerik sehingga siap dipakai sebagai sumber harga untuk QF-Lib. Keluaran sanitasi kolom memudahkan pemanggilan strategi Python di langkah berikutnya.


## 3. Strategi berbasis sinyal

Strategi yang dipakai saat ini adalah **EMA 112 momentum dengan risk-reward 1:2 berbasis ATR**:

- **Entry**: harga penutupan harian menembus EMA 112 dari bawah sementara kemiringan EMA 112 dalam `trend_lookback` hari terakhir positif. Filter slope ini membantu memilah sinyal agar hanya muncul ketika tren benar-benar menguat.
- **Exit**: kombinasi dua kondisi berbasis ATR14 yang adaptif terhadap volatilitas data OKX_BTCUSDT 1D (median ATR ≈ 3.0k, rentang 1.9k–6.5k).
  - *Stop-loss 1R*: keluar ketika penutupan ≤ harga masuk − ATR × multiplier.
  - *Take-profit 2R*: keluar ketika penutupan ≥ harga masuk + risk_reward × ATR × multiplier.
- **Konteks tambahan**: kolom `exit_flag` di sinyal/log trade menandai apakah exit terjadi karena stop atau target sehingga analisis lanjutan lebih mudah.

Strategi tambahan dapat ditaruh dalam folder `src/strategy_backtest/strategies/` dan otomatis terdaftar menggunakan `registry`.


In [4]:

available = list_strategies()
print('Strategi tersedia:', ', '.join(available))
strategy = get_strategy(STRATEGY_NAME, **STRATEGY_KWARGS)
print('Deskripsi strategi:')
print(f"- Nama: {strategy.metadata.name}")
print(f"- Deskripsi: {strategy.metadata.description}")
print(f"- Entry: {strategy.metadata.entry}")
print(f"- Exit: {strategy.metadata.exit}")
print(f"- Parameter default: {strategy.metadata.parameters}")


Strategi tersedia: ema112_atr
Deskripsi strategi:
- Nama: ema112_atr
- Deskripsi: Strategi trend-following berbasis EMA 112 dengan fokus pada momentum harga dan pengelolaan risiko fixed-R multiple menggunakan ATR harian.
- Entry: Entry long ketika harga penutupan menembus ke atas EMA 112 dari posisi di bawahnya dan EMA 112 sedang menanjak dalam jendela lookback yang ditentukan.
- Exit: Exit long ketika harga menutup di bawah level stop berbasis ATR (1R) atau mencapai target keuntungan 2R, menghasilkan rasio risk-reward asimetris 1:2.
- Parameter default: {'slow_span': 112, 'atr_window': 14, 'atr_multiplier': 1.5, 'risk_reward': 2.0, 'trend_lookback': 5}


In [ ]:

signals = strategy.generate_signals(data)
print('Kolom sinyal:', list(signals.columns))
signals.head()


## 4. Menjalankan backtest & mengekstrak hasil

In [ ]:

backtester = SignalBacktester(data=data, price_column=PRICE_COLUMN)
outputs = backtester.run(signals)

metrics = outputs.metrics
trade_summary = outputs.trade_summary
print('Metrik performa (QF-Lib / fallback):')
for key, value in metrics.items():
    if isinstance(value, (int, float, np.floating)):
        print(f'- {key}: {value:.4f}')
    else:
        print(f'- {key}: {value}')

print('Ringkasan trade:')
for key, value in trade_summary.items():
    print(f'- {key}: {value}')


In [ ]:
outputs.trades.head()

## 5. Visualisasi

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
close_prices = data[PRICE_COLUMN]
ax.plot(close_prices.index, close_prices, label='Close', color='C0', linewidth=1.1)

if 'ema_trend' in signals.columns:
    ax.plot(signals.index, signals['ema_trend'], label='EMA 112', color='C1', linewidth=1.2)
if 'stop_level' in signals.columns:
    ax.plot(signals.index, signals['stop_level'], label='Stop 1R', color='C3', linestyle='--', alpha=0.8)
if 'target_level' in signals.columns:
    ax.plot(signals.index, signals['target_level'], label='Target 2R', color='C4', linestyle='--', alpha=0.8)

long_entries = signals['long_entry'].fillna(False)
long_exits = signals['long_exit'].fillna(False)
ax.scatter(close_prices.index[long_entries], close_prices[long_entries], marker='^', color='green', label='Long Entry', zorder=5)

if 'exit_flag' in signals.columns:
    exit_flags = signals['exit_flag'].fillna('')
    stop_mask = long_exits & (exit_flags == 'stop')
    target_mask = long_exits & (exit_flags == 'target')
    remainder = long_exits & ~(stop_mask | target_mask)
    if stop_mask.any():
        ax.scatter(close_prices.index[stop_mask], close_prices[stop_mask], marker='v', color='red', label='Long Exit - Stop', zorder=6)
    if target_mask.any():
        ax.scatter(close_prices.index[target_mask], close_prices[target_mask], marker='v', color='goldenrod', label='Long Exit - Target', zorder=6)
    if remainder.any():
        ax.scatter(close_prices.index[remainder], close_prices[remainder], marker='v', color='red', label='Long Exit', zorder=6)
elif long_exits.any():
    ax.scatter(close_prices.index[long_exits], close_prices[long_exits], marker='v', color='red', label='Long Exit', zorder=6)

ax.set_title(f'{ASSET_SYMBOL} Close dengan Sinyal {STRATEGY_NAME}')
ax.set_ylabel('Harga')
ax.legend()
fig.tight_layout()
display(fig)
plt.close(fig)


In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(outputs.results.index, outputs.results['equity_curve'], color='C4', label='Equity Curve')
ax.set_title('Equity Curve Strategi')
ax.set_ylabel('Notional')
ax.legend()
fig.tight_layout()
display(fig)
plt.close(fig)

fig, ax = plt.subplots(figsize=(10, 4))
ax.hist(outputs.trades['pnl_pct'], bins=20, color='C0', alpha=0.7)
ax.set_title('Distribusi PnL per Trade')
ax.set_xlabel('PnL %')
ax.set_ylabel('Frekuensi')
fig.tight_layout()
display(fig)
plt.close(fig)


## 6. Analisis trade kalah & investigasi

In [ ]:

losing_trades = outputs.trades[outputs.trades['pnl_pct'] < 0]
print(f'Jumlah trade kalah: {len(losing_trades)} dari total {len(outputs.trades)}')
if not losing_trades.empty:
    print('Statistik ringkas trade kalah:')
    display(losing_trades[['pnl_pct', 'bars_held', 'exit_reason']].describe(include='all'))

    grouped = losing_trades.groupby('exit_reason')['pnl_pct'].agg(['count', 'mean', 'min'])
    print('Kinerja berdasarkan alasan exit:')
    display(grouped)

    context_cols = [col for col in outputs.trades.columns if col.startswith('entry_') or col.startswith('exit_')]
    if context_cols:
        print('Contoh konteks indikator pada trade kalah:')
        display(losing_trades[['trade_id', 'direction'] + context_cols].head())
else:
    print('Tidak ada trade kalah pada konfigurasi ini.')


## 7. Eksplor optimasi parameter

In [ ]:

def sweep_parameters(base_kwargs: Dict[str, float], grid: Dict[str, Iterable[float]]):
    if not grid:
        return []
    results = []
    for multiplier in grid.get('atr_multiplier', []):
        kwargs = dict(base_kwargs)
        kwargs['atr_multiplier'] = multiplier
        strat = get_strategy(STRATEGY_NAME, **kwargs)
        sig = strat.generate_signals(data)
        backtest = backtester.run(sig)
        results.append({
            'atr_multiplier': multiplier,
            'total_return': backtest.metrics.get('total_return', np.nan),
            'sharpe_ratio': backtest.metrics.get('sharpe_ratio', np.nan),
            'trades': backtest.trade_summary['total_trades'],
        })
    return results

opt_results = sweep_parameters(STRATEGY_KWARGS, OPTIMISATION_GRID)
if opt_results:
    optimisation_df = pd.DataFrame(opt_results).sort_values('sharpe_ratio', ascending=False)
    display(optimisation_df)
else:
    print('Grid optimasi kosong – lewati tahap ini.')



## 8. Dokumentasi & reusable structure

- Ganti `DATA_FILE` untuk menguji dataset TradingView lain yang memakai format serupa.
- Tambah atau modifikasi strategi di `src/strategy_backtest/strategies/`, lalu daftarkan nama modul di `registry.py`.
- Notebook ini mengandalkan `SignalBacktester` untuk menghasilkan trade log, metrik QF-Lib, serta visualisasi standar sehingga langkah analisis tetap konsisten antar strategi.
- Simpan variasi eksperimen (mis. hasil optimasi) ke file CSV tambahan bila diperlukan untuk laporan lebih lanjut.
